#Reptile Tools
##Conventions
* `rotate`, `reflect`, and `translate` occur in that order.
* `reflect` is about the y-axis.
* `childcheck = [recursive, symmetric, expansion, replacement, transformation, grid]`
* `|s1| = 1` and `|s2|` is the next smallest side, `|s3|` is next smallest...
* First tile coordinate is minimum x, then minimum y. Direction of listing is towards point with minimum x, then minimum y.
* Order amongst tile is minimum x1, then minimum y1, then minimum x2...

##Notes
* Tile starting point may fail if vertex is duplicated (i.e. not simple).

##Version History
###V0
* Initial release.

###V1
* Replaced `tiles` with `points`, `rotate`, `reflect`, and `translate`.
* Added `findtiles` to calculate the points on the tiles.
* Added `childcheck` boolean array to record which child possibilities have been explored. `[recursive, symmetric, expansion, replacement, transformation, grid]`
* Added `getrep` to assist in passing reptile data.

###V2
* Added tile vertex data under `tiles` with ordering convention.
* Removed duplicate end point from `findtiles`. `plotreptile` now handles the duplication.
* Added `pianglecheck` to check for angles of pi radians (which are invalid).
* Added duplicate checking and parent/child updates to `savereptile`. (incomplete)

##To Do List
* Create dupicate checking in `savereptile`.

In [2]:
using DataFrames, JLD, Winston

In [8]:
reptiles = load("reptileDB.jld", "reptiles");

In [120]:
function mypush!(db, n, m, ID, sides, angles, points, rotate, reflect, translate, tiles,
    classfromparents, parents, childcheck, childrensclass, children, source, notes, obsolete)
    #workaround for getting arrays into dataframes.
    
    #pushes new entry onto end of db
    push!(db, @data([n; m; ID; NA; NA; NA; NA; NA; NA; NA; NA; NA; NA; NA; NA; source;
        notes; obsolete])) #NAs are just place holders
    
    db[:sides][end] = vec(sides)
    db[:angles][end] = vec(angles)
    db[:points][end] = points
    db[:rotate][end] = vec(rotate)
    db[:reflect][end] = vec(reflect)
    db[:translate][end] = translate
    db[:tiles][end] = tiles
    db[:classfromparents][end] = classfromparents
    db[:parents][end] = parents
    db[:childcheck][end] = childcheck
    db[:childrensclass][end] = childrensclass
    db[:children][end] = children
end

mypush! (generic function with 1 method)

In [388]:
function findtiles(points, rotate, reflect, translate)
    #returns the tile points based on inputs.
    #no longer includes extra copy of first point at the final point for plotting.
    
    n = size(points)[1]
    m = length(reflect)
    output = zeros(n, 2, m)
    
    scaled = points / sqrt(m) #scaled points
    #scaled = vcat(scaled, scaled[1,:]) #duplicate first row at end for plotting
    
    for i = 1:m
        rot = [cos(rotate[i]) sin(rotate[i]); -sin(rotate[i]) cos(rotate[i])] #rotation matrix
        output[:,:,i] = scaled * rot #rotate tile
        if reflect[i] == true #reflect tile
            output[:,1,i] *= -1
        end
        output[:,:,i] .+= translate[i,:] #translate tile
    end
    
    return output
    
end

findtiles (generic function with 1 method)

In [384]:
function plotreptile(ID, tiles)
    #save a plot of the reptile given by `tiles` at the right spot

    n, m, temp = ID
    dir = pwd()*"\\images\\n="*string(n)*"\\m="*string(m)
    
    if !isdir(dir) #check for folder
        if !isdir(pwd()*"\\images\\n="*string(n))
            mkdir(pwd()*"\\images\\n="*string(n))
        end
        mkdir(dir)
    end
    
    p = plot(vcat(tiles[:,1,1], tiles[1,1,1]), vcat(tiles[:,2,1], tiles[1,2,1])) #plot first tile
    
    for i = 2:m #plot other tiles
        oplot(vcat(tiles[:,1,i], tiles[1,1,i]), vcat(tiles[:,2,i], tiles[1,2,i]))
    end
    
    setattr(p.frame, draw_nothing = true) #hide axes
      
    #force square plot
    xmin = minimum(tiles[:,1,:])
    xmax = maximum(tiles[:,1,:])
    ymin = minimum(tiles[:,2,:])
    ymax = maximum(tiles[:,2,:])
    xrange = xmax - xmin
    yrange = ymax - ymin

    if xrange >= yrange
        wid = 4096
        hgt = round(Int, 4096 * yrange / xrange)
    else
        hgt = 4096
        wid = round(Int, 4096 * xrange / yrange)
    end
    
    savefig(p, dir*"\\"*string(ID)*".pdf", 
    width = wid, height = hgt)
    savefig(p, dir*"\\"*string(ID)*".png", 
    width = wid, height = hgt)
    
end

plotreptile (generic function with 1 method)

In [385]:
function savereptile(db, n, m, ID, sides, angles, points, rotate, reflect, translate,
    classfromparents, parents, childcheck, childrensclass, children, source, notes, obsolete)
    #plots, adds to DB, and saves DB with new reptile
    
    #find ID. +1 to current number of reptiles
    num = length(db[(db[:n] .== n) & (db[:m] .== m),:ID]) + 1 #find ID. +1 to current number of reptiles
    ID = (n, m, num)
    childcheck = fill(false, 6)
    childrensclass = Array{AbstractString,1}[]
    children = Array{Tuple{Int64,Int64,Int64},1}[]
    
    tiles = ordertiles(findtiles(points, rotate, reflect, translate))
    
    mypush!(db, n, m, ID, sides, angles, points, rotate, reflect, translate, tiles,
    classfromparents, parents, childcheck, childrensclass, children, source, notes, obsolete) #update db
    
    plotreptile(ID, tiles)
    
    save("reptileDB.jld", "reptiles", db)
    
    return ID
    
end 

savereptile (generic function with 1 method)

In [386]:
function getrep(db, ID)
    #returns the row in the database with the corresponding `ID`.
    
    rep = db[db[:ID] .== ID,:]
    if size(rep)[1] != 1
        error("ID does not exist or is not unique.")
    end
    return rep
end

getrep (generic function with 1 method)

In [4]:
function initreptiles()
    #reinitialized DB
    reptiles = DataFrame()
    reptiles[:n] = Int64[] #number of sides
    reptiles[:m] = Int64[] #number of tiles
    reptiles[:ID] = Tuple{Int64,Int64,Int64}[] #n,m,x where x is such that ID is unique
    reptiles[:sides] = Array{Float64,1}[] #side lengths
    reptiles[:angles] = Array{Float64,1}[] #angles
    reptiles[:points] = Array{Float64,2}[] #verticies of original
    reptiles[:rotate] = Array{Float64,1}[] #rotation of each tile (radians)
    reptiles[:reflect] = Array{Bool,1}[] #is each tile a reflection?
    reptiles[:translate] = Array{Float64,2}[] #the translation of each tile
    reptiles[:tiles] = Array{Float64,3}[] #tile verticies
    reptiles[:classfromparents] = Array{AbstractString,1}[] #class of this reptile (how it was derived from parent).
    reptiles[:parents] = Array{Tuple{Int64,Int64,Int64},1}[]
    reptiles[:childcheck] = Array{Bool,1}[] #which possibilties for children have been explored.
                                            #[recursive, symmetric, expansion, replacement, transformation, grid]
    reptiles[:childrensclass] = Array{AbstractString,1}[] #class of children
    reptiles[:children] = Array{Tuple{Int64,Int64,Int64},1}[]
    reptiles[:source] = AbstractString[] #citation
    reptiles[:notes] = AbstractString[]
    reptiles[:obsolete] = Bool[]
    
    return reptiles
end

initreptiles (generic function with 1 method)

In [90]:
function ordertiles(tiles, digits = 9)
    #orders each tile starting with min x, then min y. Heads towards min x, then min y.
    #orders amongst tiles by min x, then min y. proceeds to next point in case of a tie.
    #sorts within and among tiles
    
    roundedtiles = round(tiles, digits)
    
    #order each tile
    for i = 1:size(tiles)[3] #for each tile
        
        #find starting point (duplicate points impossible since tile is simple)
        xmins = roundedtiles[:,1,i] .== minimum(roundedtiles[:,1,i]) #is each x value the minimum?
        if sum(xmins) > 1 #minimum is not unique
            yorder = sortperm(roundedtiles[:,2,i], rev=true) #reverse order of y values
            startind = indmax(xmins .* yorder) #starting index. xmin falses will become zero.
        else #minimum is unique
            startind = indmin(roundedtiles[:,1,i])
        end

        #reorder to begin at starting point
        tiles[:,:,i] = circshift(tiles[:,:,i], [-startind + 1,0 ,0])
        roundedtiles[:,:,i] = round(tiles[:,:,i], digits) #update new tile location
        
        #flip order if necessary
        if roundedtiles[2,1,i] > roundedtiles[end,1,i] #headed wrong direction
            tiles[:,:,i] = circshift(flipdim(tiles[:,:,i], 1), [1,0,0]) #flip order then shift to fix starting point
        elseif roundedtiles[2,1,i] == roundedtiles[end,1,i] #tie
            if roundedtiles[2,2,i] > roundedtiles[end,2,i] #y-component is tiebreak
                tiles[:,:,i] = circshift(flipdim(tiles[:,:,i], 1), [1,0,0]) #flip order then shift to fix starting point
            end
        end #else order was already correct
            
    end
    
    #order amongst tiles
    tile = sorttiles(tiles, 1, digits)
    
    return tiles
    
end
        

ordertiles (generic function with 2 methods)

In [91]:
function sorttiles(tiles, depth, digits = 9)
    #recursively sorts a tile until it is ordered by x1, then y1, then x2, then y2...
    #sorts amongst tiles
    
    #init
    row = ceil(Int, depth/2)
    col = mod1(depth,2)
    roundtiles = round(tiles, digits)
    
    #sort
    tileorder = sortperm(vec(roundtiles[row, col,:]))
    tiles[:, :, :] = tiles[:, :, tileorder]
    roundtiles = round(tiles, digits) #reround for new order
    
    #check if next level needs to be sorted
    for i = 1:size(tiles)[3] - 1 #each tile
        
        #count number of identical values in a row
        j = 0 #number of identical values in a row
        while roundtiles[row, col, i] == roundtiles[row, col, i + j + 1] #compare to next number
            j += 1 #increment
            if i + j + 1 > size(tiles)[3] #end of array
                break
            end
        end

        #sort next level if necessary
        if j > 0 #duplicates found
            tiles[:,:,i:i+j] = sorttiles(tiles[:,:,i:i+j], depth + 1, digits)
        end
        
        #move to next
        i += j       
    end
    
    return tiles
end

sorttiles (generic function with 2 methods)

In [11]:
function pianglecheck(tile, digits = 9)
    #delete any verticies that fall on a straight line (pi radians)
    
    difftile = round(diff([tile; tile[1,:]]), digits) #change in x and y. duplicated points should be impossible.
    slopes = difftile[:,2] ./ difftile[:,1] #slopes
    
    equalslopes = slopes .== circshift(slopes, 1) #compare each side with slope of next
    tile = tile[!equalslopes,:] #remove invalid points
    
    return tile
    
end

pianglecheck (generic function with 2 methods)